In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_colwidth', None)
import re
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

сначала делаю парсинг всех ссылок на вакансии и добавляю их в массив

In [ ]:
url_list = []

# Цикл для перебора страниц
for page_number in tqdm(range(1, 167)):  # Замените 167 на количество страниц, которое вам нужно
    #url = f'https://russia.superjob.ru/resume/search_resume.html?keywords%5B0%5D%5Bkeys%5D="data%20science"&sbmit=1&page={page_number}'
    url = f'https://russia.superjob.ru/resume/it-internet-svyaz-telekom/analitika/?page={page_number}'

    r = requests.get(url, verify=False, timeout=10)
    soup = BeautifulSoup(r.content, 'html.parser')
    divs = soup.find_all('div', attrs={'class': 'f-test-search-result-item'})

    for div in divs[:-1]:
        resume_url = "https://russia.superjob.ru" + div.find_all('a', class_='_1IHWd')[0].get('href')
        if len(resume_url) > 55:
            url_list.append(re.sub(r'search.*?&', '', resume_url))

100%|██████████| 166/166 [04:33<00:00,  1.65s/it]


In [ ]:
len(url_list)

5026

In [ ]:
url_list[1:167]

['https://russia.superjob.ru/targetirovannaya_reklama/?utm_source=superjob&utm_campaign=desktop_promo_banner',
 'https://russia.superjob.ru/resume/sistemnyj-analitik-54359240.html?search_id=98d3108a-6481-11ee-a48c-cebcc7621d54',
 'https://russia.superjob.ru/resume/analitik-dannyh-41665651.html?search_id=98d3108a-6481-11ee-a48c-cebcc7621d54',
 'https://russia.superjob.ru/resume/analitik-54468122.html?search_id=98d3108a-6481-11ee-a48c-cebcc7621d54',
 'https://russia.superjob.ru/resume/data-analyst-54400034.html?search_id=98d3108a-6481-11ee-a48c-cebcc7621d54',
 'https://russia.superjob.ru/resume/analitik-dannyh-54474196.html?search_id=98d3108a-6481-11ee-a48c-cebcc7621d54',
 'https://russia.superjob.ru/resume/junior-data-analyst-54470926.html?search_id=98d3108a-6481-11ee-a48c-cebcc7621d54',
 'https://russia.superjob.ru/resume/analitik-54485638.html?search_id=98d3108a-6481-11ee-a48c-cebcc7621d54',
 'https://russia.superjob.ru/resume/stazher-analitik-39898239.html?search_id=98d3108a-6481-11e

создаю функцию для парсинга которая на вход будет принимать только URL

In [ ]:
def get_resume_data(url):
    r = requests.get(url, verify=False, timeout=120)
    soup1 = BeautifulSoup(r.content, 'html.parser')

    quote = {}
    divs1 = soup1.find('div', class_='_31epc')
    #print(divs1)
    try:
        quote['Должность'] = divs1.find('div', class_='f-test-block-position').find('span', class_='_3ExbL _1Tz2P _1BJoX _2iNjB').text
    except AttributeError:
        quote['Должность'] = 'Нет данных'


    try:
        quote['Зарплата'] = divs1.find('span', class_='_1Tz2P _1BJoX _2EuTH').text
    except AttributeError:
        quote['Зарплата'] = 'Нет данных'


    try:
        text = divs1.find('span', class_='_2fdQP _1BJoX inakc _2EuTH').text
        quote['Местоположение'] = [word for word in text.split(' ') if word[0].isupper()][0].split(',')[0]

    except AttributeError:
        quote['Местоположение'] = 'Нет данных'


    try:
        info_person = divs1.find_all('div', class_='_31epc')
        if len(info_person) == 0:
             quote['Тип занятости'] = 'Нет данных'
        else:
            info_person = info_person[1]
            quote['Тип занятости'] = info_person.find('span', class_='_2fdQP _1BJoX inakc _2EuTH').text
    except AttributeError:
        quote['Тип занятости'] = 'Нет данных'


    try:
        """
        info_person_list = divs1.find_all('div', class_='_31epc')
        if len(info_person_list) > 1:
            info_person = info_person_list[1]
            quote['Тип занятости'] = info_person.find('span', class_='_2fdQP _1BJoX inakc _2EuTH').text
        else:
            continue
        """
        # Обработка случая, когда элемент не найден
            #quote['Тип занятости'] = "Нет информации"
        info_person = divs1.find('div', class_='_3eSqg N4A69')
        text = info_person.find('span', class_='_1VXtA _1jDNY _2owJj cfIQS _2fdQP _1BJoX inakc GQbw5').text

        ages = [age for age in text.split(' ') if len(age) != 0 and age[0].isdigit()]
        if ages:
            quote['Возраст'] = ages[0]
        else:
            quote['Возраст'] = 'Нет данных'
    except AttributeError:
        quote['Возраст'] = 'Нет данных'
    try:
        info_person = divs1.find('div', class_='_3eSqg N4A69')
        text = info_person.find('span', class_='_1VXtA _1jDNY _2owJj cfIQS _2fdQP _1BJoX inakc GQbw5').text
        if 'родился' in text or 'готов' in text:
            quote['Пол'] = 'М'
        else:
            quote['Пол'] = 'Ж'
    except AttributeError:
        quote['Пол'] = 'Нет данных'

    try:
        info_persons = divs1.find('div', class_='f-test-block-assignment')
        exp = info_persons.find('h2', '_-99Mm _2IT_8 _1BJoX inakc GQbw5')
        if  exp.text.split(' ')[0] == 'Опыт':
            quote['Опыт работы'] = exp.text[12:].replace('\xa0', ' ')
        else:
            quote['Опыт работы'] = 'Нет данных'
    except AttributeError:
        quote['Опыт работы'] = 'Нет данных'
    try:
        info_persons = divs1.find('div', class_='f-test-block-assignment')
        exp = info_persons.find('div', '_2YEGx _3VcRo _1CqoT')

        quote['Последнее место работы'] = exp.find('h3', class_='_2J46f _1BJoX inakc _2EuTH _30f1O').text + ' ' + exp.find('span',
                                                                                                                    class_='_2fdQP _1BJoX _1ux5Q _2EuTH _30f1O').text

    except AttributeError:
        quote['Последнее место работы'] = 'Нет данных'

    try:
        lin = []
        info_persons = divs1.find('div', class_='f-test-block-assignment')
        links = list(map(lambda x: x['href'] if ('github' in x['href']) else '' , info_persons.find_all('a')))
        for i in links:
            if i == '':
                continue
            else:
                lin.append(i)
        if lin:
            quote['Ссылки_проекты'] = lin
        else:
            quote['Ссылки_проекты'] = 'Нет данных'

    except AttributeError:
        quote['Ссылки_проекты'] = 'Нет данных'

    try:
        info_persons = divs1.find('div', class_='f-test-block-account_balance')
        exp = info_persons.find('span', '_2fdQP _1BJoX _1ux5Q _2EuTH _2B9p-')
        quote['Образование'] = exp.text
    except AttributeError:
        quote['Образование'] = 'Нет данных'


    try:
        data_list = []
        qwe = divs1.find_all('div', class_='_3VMkc _1CS2f xF3In _254eA _2-THL _1vMSo')[-1].find('ul', class_='_1VUIv _1tvYv _24ay7 LS7Gy _2DqUc')
        for li in qwe.find_all('li', class_='pmZgY'):

            item = li.find('span', class_='jj0ch _2fdQP _1BJoX inakc GQbw5')
            if item:
                data_list.append(item.text)
        quote['Профессиональные навыки'] = ', '.join(data_list)
    except AttributeError:
        quote['Профессиональные навыки'] = 'Нет данных'


    try:
        extra_info = {}
        extra = divs1.find('div', class_='f-test-block-feedback').find_all('div', class_='_3VcRo _1CqoT')[-1]
        for i in extra.find_all('div', class_='pmZgY', recursive=False):
            quote[i.find('h4', class_='_1jYKu _1BJoX inakc _2EuTH').text + '_Обо мне'] = i.find('span', class_='_2BpzE _1W4TG _1BJoX inakc _2EuTH _30f1O')


    except:

        pass


    try:
        quote['Иностранные языки'] = divs1.find('div', class_='f-test-block-sort_by_alpha').find('span').text
    except AttributeError:
        quote['Иностранные языки'] = 'Нет данных'


    return quote

In [ ]:
df = pd.DataFrame()
for res_url in tqdm(url_list):
    try:
        resume_data = get_resume_data(res_url)
        if resume_data:
            df = df.append(resume_data, ignore_index=True)
    except Exception as e:
        print(f"Ошибка при обработке резюме {res_url}: {e}")

 10%|▉         | 497/5026 [09:52<1:47:40,  1.43s/it]

Ошибка при обработке резюме https://russia.superjob.ru/resume/analitik-dannyh-54457973.html?search_id=aadd786a-6481-11ee-9ede-41c65212ea61: list index out of range


 79%|███████▉  | 3981/5026 [1:36:04<24:00:45, 82.72s/it] 

Ошибка при обработке резюме https://russia.superjob.ru/resume/menedzher-po-produktu-53638609.html?search_id=1b9e26d0-6482-11ee-a740-3270ebeeaba6: HTTPSConnectionPool(host='russia.superjob.ru', port=443): Read timed out.


100%|██████████| 5026/5026 [1:55:20<00:00,  1.38s/it]   


In [ ]:
df

,Должность,Зарплата,Местоположение,Тип занятости,Возраст,Пол,Опыт работы,Последнее место работы,Ссылки_проекты,Образование,Профессиональные навыки,Дополнительные сведения:_Обо мне,Иностранные языки,Профессиональные навыки:_Обо мне
0,"Руководитель, ведущий специалист информационно-аналитического отдела, политолог, исследователь общественно-политических и социально-экономических процессов, специалист в сфере управления проектами",По договорённости,Новомосковск,не указано,53,М,23 года и 6 месяцев,"Исследователь общественно-политических и социально-экономических процессов Независимый аналитик на удаленном режиме, Новомосковск (Тульская область)",Нет данных,Кандидат наук,Нет данных,"[Моя колонка в Центре политического анализа: , [https://centerforpoliticsanalysis.ru/experts/view/id/250], \n\nМоя Дзен-визитка: , [https://dzen.ru/a/Y-6YuNq05yNPz7gi], \n\nТелеграм-каналы, ссылки-приглашения:\nПОЛИТлента: , [https://t.me/U1zqLHndIzS56xZ8], \nЭКОлента: , [https://t.me/SXRlqfgN2ySKgT], \nКИБЕРлента: , [https://t.me/QjVYOX0_dX0yMjYy]]",Английский язык — базовый,NaN
1,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,NaN,Нет данных,NaN
2,Системный аналитик,По договорённости,Москва,полная занятость,Нет данных,Ж,23 года и 9 месяцев,"Администратор сайта, системный администратор ООО ""СпецЭнергоСистемы"", Москва",Нет данных,Среднее специальное образование,"Работа с большим объемом информации, Аналитический склад ума, Составление спецификаций, eEPC, Системный анализ, Разработка технических заданий, Моделирование бизнес-процессов (IDEF, BPMN, UML), Figma,, SQL и администрирование баз данных, Маркетинговый анализ",NaN,Английский язык — технический,NaN
3,Аналитик данных,80 000 ₽,Москва,полная занятость,69,М,50 лет и 1 месяц,"Инженер 1 категории АО «РАДИЙ ТН» , Москва",Нет данных,Высшее образование,Нет данных,NaN,Английский язык — технический,"[Владею профессиональными навыками:\n\nПланирования, административного управления, организации работы коллектива (от 10 до 250 чел. в качестве проректора СВФУ – 17 000 чел.); подбора кадров и их обучения; алгоритмизацией процессов управления.\nРуководства проектами в области автоматизации и телекоммуникаций; разработки бизнес-планов в области телекоммуникаций и информационных технологий.\nРазвертывания компаний в области информационных технологий и производства аэросъемочных работ по территориально-протяженным объектам электрических сетей.\nОрганизации эксплуатации и применения технических средств и комплексов программного обеспечения (АСУ, PC, LAN, WAN, TV, САПР AutoCAD);\nОрганизации разработки и эксплуатации территориальных мониторинговых информационных систем на основе спутниковых данных, данных аэросъемки, GPS, метеорологических систем и других источников, в том числе с использованием геоинформационных технологий.\nСбора и аналитической обработки информации с квалификацией аналитика данных, имею учебную практику анализа данных и машинного обучения в сфере анализа больших данных\nРаботы на ПК (WINDOWS, WORD, EXEL, ACCESS, MS PROJECT, INTERNET, графические средства AutoCAD, ГИС-пакеты и др.);\nВождения автомобиля (права категории «В», практический стаж 30 лет).\n\nЛичные качества\n\nУмение работать в команде, коммуникабельность, уравновешенность, профессионализм, творческий подход и упорство в достижении цели.]"
4,Аналитик,По договорённости,Москва,полная занятость,43,Ж,14 лет и 3 месяца,"Трейд-маркетолог Виником, Москва",Нет данных,Нет данных,Нет данных,NaN,Английский язык — разговорный,"[- Работа с большим объемом данных;\n- Опыт разработки и поддержания системы отчетности подразделения;\n- Владею на хорошем уровне Word, Excel (включая кубы OLAP, сводные таблицы, ВПР), Power Point, 1С, Bitrix24, др.\n- План / факт продаж;\n- Многозадачность;\n- Структурное мышление.]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019,Junior systems analyst,75 000 ₽,Москва,не указано,37,М,14 лет и 6 месяцев,"Специалист по тестированию веб

In [ ]:
df.to_csv('resums_superjob_analytics.csv', index=False)